In [0]:
import pandas as pd 
import numpy as np

In [0]:
data = pd.read_csv('/content/Churn_Modelling.csv')

In [6]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [0]:
x = data.iloc[:,3:13].values
y = data.iloc[:, 13].values

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('Geography', OneHotEncoder(), [1])], remainder = 'passthrough')
x = ct.fit_transform(x)
labelencoder_x = LabelEncoder()
x[:, 4] = labelencoder_x.fit_transform(x[:, 4])
x = np.array(x, dtype = np.float64)

In [0]:
# droping first column to avoid dummy trap
x = x[:, 1:]

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [0]:
# standardizing features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [13]:
from keras.models import Sequential
from keras.layers import Dense
# initializing ANN with input layer and first hidden layer
def classifier():
    
    # initializing ANN
    model = Sequential()
    model.add(Dense(activation = 'relu', units = 6, kernel_initializer = 'uniform', input_dim = 11))
    model.add(Dense(activation = 'relu', units = 6, kernel_initializer = 'uniform'))
    model.add(Dense(activation = 'sigmoid', units = 1, kernel_initializer = 'uniform'))    
    
    # compiling ANN
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics= ['accuracy'])
    
    return model

Using TensorFlow backend.


In [14]:
model = classifier()

# fitting the training set
model.fit(x_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 154us/step - loss: 0.4864 - accuracy: 0.7958
Epoch 2/100
8000/8000 [==============================] - 1s 116us/step - loss: 0.4280 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 114us/step - loss: 0.4230 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 114us/step - loss: 0.4193 - accuracy: 0.8134
Epoch 5/100
8000/8000 [==============================] - 1s 117us/step - loss: 0.4168 - accuracy: 0.8249
Epoch 6/100
8000/8000 [==============================] - 1s 119us/step - loss: 0.4147 - accuracy: 0.8300
Epoch 7/100
8000/8000 [==============================] - 1s 119us/step - loss: 0.4135 - accuracy: 0.8305
Epoch 8/100
8000/8000 [==============================] - 1s 116us/step - loss: 0.4122 - accuracy: 0.8310
Epoch 9/100
8000/8000 [==============================] - 1s 116us/step - loss: 0.4113 - accuracy: 0.8320
Epoch 10/100
8000/8000 [==============================]

In [0]:
y_predic = model.predict(x_test)   #predicting results

In [16]:
y_predi_bin = (y_predic > 0.5)
y_predi_bin

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_predi_bin, y_test)             # predicting power

array([[1546,  277],
       [  49,  128]])

In [23]:
# checking accuracy
(1546 + 128)/2000

0.837

# Model Evaluation using k-fold cross-validation

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [0]:
model = KerasClassifier(build_fn = classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = model, X = x_train, y = y_train, cv = 10, n_jobs = -1)

In [22]:
mean = accuracies.mean()
variance = accuracies.std()
print('Accuracy: {:.2f}\nVariance: {:.2f}'.format(mean,variance))

Accuracy: 0.84
Variance: 0.02
